# Autoencoder + One Class SVM Ensemble Model

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
import pandas as pd
import numpy as np

# Load and preprocess the data
data = pd.read_csv('../Dataset/Aggregated_Sleep.csv')
data = data.drop(['patient_id', 'window_start'], axis=1)

# Drop rows with NaNs
data = data.dropna()

X = data.drop('agitation', axis=1).values
y = data['agitation'].values

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data, focusing only on normal data for training the Autoencoder
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y == 0, test_size=0.2, random_state=42)
X_train_normal = X_train[y_train]

# Define the Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, n_features):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n_features, 10),
            nn.ReLU(),
            nn.Linear(10, 2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 10),
            nn.ReLU(),
            nn.Linear(10, n_features),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Training the Autoencoder
autoencoder = Autoencoder(X_train_normal.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.01)
train_loader = DataLoader(TensorDataset(torch.tensor(X_train_normal).float()), batch_size=32, shuffle=True)

def train_autoencoder(model, loader, epochs=50):
    model.train()
    for epoch in range(epochs):
        for data in loader:
            data = data[0]  # unpack data
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, data)
            loss.backward()
            optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

train_autoencoder(autoencoder, train_loader)

# Extract features for training and testing
autoencoder.eval()
with torch.no_grad():
    X_train_encoded = autoencoder.encoder(torch.tensor(X_train).float()).numpy()
    X_test_encoded = autoencoder.encoder(torch.tensor(X_test).float()).numpy()

# Train One-Class SVM on the encoded features
oc_svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
oc_svm.fit(X_train_encoded[y_train])  # Train only on normal data

# Predictions and Evaluation
from sklearn.metrics import classification_report

y_pred_train = oc_svm.predict(X_train_encoded)
y_pred_test = oc_svm.predict(X_test_encoded)
y_pred_test = np.where(y_pred_test == 1, 0, 1)  # Converting from SVM labels to anomaly labels

print("Classification Report (Test Set):")
print(classification_report(y_test, y_pred_test))

Epoch 1/50, Loss: 1.1442
Epoch 11/50, Loss: 0.5622
Epoch 21/50, Loss: 0.4725
Epoch 31/50, Loss: 0.8408
Epoch 41/50, Loss: 0.7065
Classification Report (Test Set):
              precision    recall  f1-score   support

       False       0.04      1.00      0.08         7
        True       1.00      0.15      0.25       178

    accuracy                           0.18       185
   macro avg       0.52      0.57      0.17       185
weighted avg       0.96      0.18      0.25       185

